# 🎙️ Amir Voice Test - Ultimate TTS Playground

This notebook consolidates all Text-to-Speech engines for the **Su6i Yar** project.

## 🛠️ Prerequisites & Setup
Before running this notebook, ensure you have the following installed:

### 1. Python Libraries
Run the following command in your terminal:
```bash
pip install edge-tts sherpa-onnx soundfile numpy requests
```

### 2. System Tools (FFmpeg)
Required for audio conversion (WAV -> MP3).
- **Mac:** `brew install ffmpeg`
- **Linux:** `sudo apt install ffmpeg`
- **Windows:** Download from ffmpeg.org and add to PATH.

### 3. Local Model (Sherpa)
Ensure the model file exists at `models/fa_IR-mana-medium-fixed.onnx`.

---


## 1. Setup & Imports
Run this cell first to load necessary libraries.

In [ ]:
import os
import sys
import time
import wave
import struct
import numpy as np
import soundfile as sf
import json
import requests
from IPython.display import Audio, display, Markdown

# Try importing Sherpa (Local Engine)
try:
    import sherpa_onnx
    SHERPA_AVAILABLE = True
    print("✅ sherpa-onnx found.")
except ImportError:
    SHERPA_AVAILABLE = False
    print("❌ sherpa-onnx not found. (Local TTS will be disabled)")

# Constants for Local Model
MODEL_PATH = "models/fa_IR-mana-medium-fixed.onnx"
ORIGINAL_CONFIG = "models/fa_IR-mana-medium.onnx.json"
TOKENS_PATH = "models/tokens.txt"
ESPEAK_DATA = "/opt/homebrew/share/espeak-ng-data"

# 🌍 Test Sentences
TEST_PHRASES = {
    "fa": "سلام امیرجان. این تست صدای مانا روی مک‌بوک شماست. سرعت را ببینید!",
    "en": "Hello Amir! This is a test of the Persian model speaking English.",
    "fr": "Bonjour! C'est un test du modèle persan parlant français.",
    "ko": "안녕하세요! 이것은 페르시아어 모델이 한국어를 말하는 테스트입니다."
}

def convert_wav_to_mp3(wav_path, mp3_path):
    """Helper to convert WAV to MP3 using ffmpeg for user request"""
    if os.path.exists(mp3_path):
        os.remove(mp3_path)
    cmd = f'ffmpeg -y -v error -i "{wav_path}" -acodec libmp3lame -q:a 2 "{mp3_path}"'
    os.system(cmd)
    return mp3_path

## 2. 🧠 Engine: EdgeTTS (Farid)
**Model:** Microsoft Farid (Online)  
**Quality:** Standard, slightly robotic

In [ ]:
def run_edgetts_tests():
    print("--- 🧠 Running Microsoft EdgeTTS Tests (All Voices) ---")
    
    # Voices to test per language
    VOICES = {
        "fa": ["fa-IR-FaridNeural", "fa-IR-DilaraNeural"],
        "en": ["en-US-JennyNeural", "en-US-GuyNeural"],
        "fr": ["fr-FR-DeniseNeural", "fr-FR-HenriNeural"],
        "ko": ["ko-KR-SunHiNeural", "ko-KR-InJoonNeural"]
    }
    
    for lang, text in TEST_PHRASES.items():
        display(Markdown(f"### 🏳️ {lang.upper()}"))
        target_voices = VOICES.get(lang, [])
        
        for voice in target_voices:
            display(Markdown(f"**🗣️ Voice:** `{voice}`"))
            mp3_filename = f"edge_{lang}_{voice}.mp3"
            
            cmd = f'edge-tts --text "{text}" --voice {voice} --write-media {mp3_filename}'
            
            start = time.time()
            res = os.system(cmd)
            if res == 0:
                 print(f"✅ Generated in {time.time()-start:.2f}s")
                 display(Audio(mp3_filename, autoplay=False))
            else:
                print(f"❌ Failed for {voice}")
            print("-" * 10)
        print("=" * 30)

run_edgetts_tests()

## 3. 🧪 Engine: Local Piper (Sherpa-ONNX)
**Model:** Mana (Medium)  
**Speed:** Real-time on CPU (M1/M2)  
**Quality:** High

In [ ]:
# Initialize Sherpa Engine
if SHERPA_AVAILABLE and os.path.exists(MODEL_PATH):
    print("🚀 Initializing Sherpa-ONNX...")
    
    # Generate tokens if missing
    if not os.path.exists(TOKENS_PATH) and os.path.exists(ORIGINAL_CONFIG):
        with open(ORIGINAL_CONFIG, "r", encoding="utf-8") as f:
            data = json.load(f)
        id_map = data.get("phoneme_id_map", {})
        with open(TOKENS_PATH, "w", encoding="utf-8") as f:
            for symbol, ids in id_map.items():
                if ids:
                    f.write(f"{symbol} {ids[0]}\n")
        print("✅ tokens.txt generated.")

    try:
        config = sherpa_onnx.OfflineTtsConfig(
            model=sherpa_onnx.OfflineTtsModelConfig(
                vits=sherpa_onnx.OfflineTtsVitsModelConfig(
                    model=MODEL_PATH,
                    tokens=TOKENS_PATH,
                    data_dir=ESPEAK_DATA,
                    noise_scale=0.667,
                    length_scale=1.0,
                    noise_scale_w=0.8,
                ),
                provider="cpu",
                num_threads=1,
                debug=False
            )
        )
        tts_sherpa = sherpa_onnx.OfflineTts(config)
        print("✅ Sherpa Engine Ready!")
    except Exception as e:
        print(f"❌ Initialization Failed: {e}")
        tts_sherpa = None
else:
    print("⚠️ Sherpa engine skipped (missing files or lib).")
    tts_sherpa = None

def run_sherpa_tests():
    if not tts_sherpa:
        print("❌ Engine not loaded.")
        return
        
    print("--- 🧪 Running Sherpa Multi-Lang Tests ---")
    for lang, text in TEST_PHRASES.items():
        display(Markdown(f"**{lang.upper()}:** `{text}`"))
        
        start = time.time()
        try:
            audio = tts_sherpa.generate(text, sid=0, speed=1.0)
            duration = time.time() - start
            
            # Save as WAV first
            wav_filename = f"sherpa_{lang}.wav"
            mp3_filename = f"sherpa_{lang}.mp3"
            
            if len(audio.samples) > 0:
                sf.write(wav_filename, audio.samples, audio.sample_rate)
                
                # Convert to MP3
                convert_wav_to_mp3(wav_filename, mp3_filename)
                
                print(f"⚡ Generated in {duration:.3f}s -> {mp3_filename}")
                display(Audio(mp3_filename, autoplay=False))
            else:
                print(f"⚠️ No audio generated for {lang}")
                
        except Exception as e:
            print(f"❌ Failed for {lang}: {e}")
        
        print("-" * 30)

run_sherpa_tests()

## 4. ☁️ Engine: Datacula (Amir)
**Model:** Amir  
**Type:** Online API  
**Latency:** Depends on network

In [ ]:
def run_datacula_tests():
    print("--- ☁️ Running Datacula Multi-Lang Tests ---")
    url = "https://tts.datacula.com/api/tts"
    
    for lang, text in TEST_PHRASES.items():
        display(Markdown(f"**{lang.upper()}:** `{text}`"))
        params = {"text": text, "model_name": "امیر"}
        
        start = time.time()
        try:
            resp = requests.get(url, params=params, timeout=10)
            if resp.status_code == 200:
                print(f"✅ Generated in {time.time()-start:.2f}s")
                
                # Save
                wav_filename = f"datacula_{lang}.wav"
                mp3_filename = f"datacula_{lang}.mp3"
                
                with open(wav_filename, "wb") as f:
                    f.write(resp.content)
                
                convert_wav_to_mp3(wav_filename, mp3_filename)
                display(Audio(mp3_filename, autoplay=False))
            else:
                print(f"❌ Failed: {resp.status_code}")
        except Exception as e:
            print(f"❌ Error: {e}")
        print("-" * 30)

# run_datacula_tests() # Uncomment to run (requires internet & API key validity)

---